# Requests et BeautifulSoup

## Installation Des Packages

In [1]:
# Si nous n'avez pas encore installé ces packages il faut enlever les les lignes suivantes en commentaire etles executer
# !pip install requests --upgrade --quiet
# !pip install beautifulsoup4 --upgrade --quiet
# !pip install pandas --quiet

## Importation des Packages

In [2]:
import requests
from random import randint
from time import sleep
from bs4 import BeautifulSoup
import pandas as pd

## Comment sa fonctionne

La première étape consiste à utiliser le package <code>**Requests**</code> pour télécharger le code HTML de notre page Web (à l'aide de la fonction <code>**requests.get**</code>) qui donne un objet de réponse. Nous pouvons voir si nous pouvons télécharger la page Web ou non en vérifiant la réponse. Statut de notre objet. (Il devrait être compris entre <code>**200 et 299**</code>). Ensuite, nous allons convertir cet objet de réponse en un objet <code>**BeautifulSoup**</code> en utilisant le constructeur <code>**BeautifulSoup()**</code>. Nous utiliserons cet objet pour inspecter notre document et extraire les données souhaitées. <code>**BeautifulSoup**</code> nécessite un argument supplémentaire appelé <code>**parser**</code>. (En bref, **BeautifulSoup** peut également être utilisé pour extraire des informations d'autres langages de balisage. Le **parser** par défaut est <code>**"html.parser"**</code>). Trouvez ci-dessous une fonction d'exemple indépendante qui effectuera la tâche ci-dessus: 
* de téléchargement des pages, 
* de vérification des réponses
d'analyse à l'aide de <code>**BeautifulSoup()**</code>.

In [6]:
def get_item_page(items_url):
    # télécharger la page
    response = requests.get(items_url)
    # vérifier le succès de réponse
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(items_url))
    # Parser la réponse à l'aide de beaufifulSoup
    doc = BeautifulSoup(response.text, 'html.parser')
    return doc

## Scraper la liste des articles de notre site Web cible
Dans notre cas, nous allons scraper https://www.alibaba.com/trade/search?fsb=y&IndexArea=product_en&CatId=&SearchText=Inflight+Items&viewtype=G&tab={page} Nous obtiendrons une liste des produits en vol. Nous obtiendrons un **titre d'article principal**, un **prix**, **des notes** et une **URL** de page pour chaque article. Pour **chaque article**, nous obtiendrons plus de détails sur le produit à partir de la page de l'article (si nécessaire). Nous allons créer un dataframe de Pandas et un fichier **Json** avec les détails de l'élément. Pour chaque article, nous créerons également un fichier **CSV** au format indicatif suivant:

nom, prix, URL
Article JUNIO Fleur Préservée Avec Boîtes, 5.50 - 6.50, https://www.alibaba.com/product-detail/Item-JUNIO-Preserved-Flower-With-Boxes_62339028105.html?s=p

In [7]:
url = "https://www.alibaba.com/trade/search?fsb=y&IndexArea=product_en&CatId=&SearchText=Inflight+Items&viewtype=G&tab=%7Bpage%7D"

In [13]:
doc = get_item_page(url)

## Comprendre les bases de BeautifulSoup

**BeautifulSoup** a plusieurs méthodes pour extraire des informations de notre document analysé. La façon la plus simple de rechercher une balise dans notre document est d'appeler directement le nom de la balise que nous recherchons. Par exemple, si je veux la balise &lt;title&gt;, j'appellerai <code>doc.title</code>. Si je passe <code>doc.a</code>, il renverra l'occurrence de la première balise &lt;a&gt; présente dans mon document. Et si je voulais toutes les balises &lt;a&gt; ? Nous devons utiliser : « La méthode <code>find_all()</code> ».

## BeautifulSoup: (.find_all() method)

><code>doc.find_all(tag_name, attributes, limit, string, recursive=True)</code>:

* tag_name: Le nom des éléments **HTML** ex: &lt;a&gt;, &lt;tr&gt;
* attributes: un dict contenant les attributs d'une balise ex: {“class”: “abcabc”}
* limit: un nombre pour limiter le nombre de balises sous cette correspondance.
* text: une chaîne pour correspondre au contenu d'un élément 
* recursive: Par défaut, Beautiful Soup recherche tous les éléments enfants. Ainsi, le paramètre recursive = False limitera la recherche au premier élément trouvé et à son enfant uniquement.

In [36]:
tousLesDiv = doc.find_all('div')

In [37]:
len(tousLesDiv)

253

In [38]:
tousLesDivAvecClass = doc.find_all('div',{'class': 'sc-hd'})

In [29]:
len(tousLesDivAvecClass)

1

In [40]:
sixPremierDiv = doc.find_all('div', limit=6)
len(sixPremierDiv)

6

## BeautifulSoup: (.find() method)

Une autre méthode répandue est, qui fonctionne comme mais ne renvoie que la première occurrence de la balise que nous avons demandée

Dans notre recherche spécifique, nous avons des attributs dont la valeur est nulle. Donc, tout d'abord, nous organisons/analysons les données par rapport à la balise de liste d'éléments. En recherchant sur le site Web à l'aide de l'option Inspecter (clic droit sur les éléments), nous avons constaté que les informations sont disposées dans la balise div avec class = "organic-gallery-offer-outter J-offer-wrapper".

![image](images/1.png)

Le code du Web Scraping dépend de la structure de la page Web. Donc, si la conception change, votre code a également besoin d'une mise à jour. Nous utilisons une boucle for et une méthode append pour organiser les données en item_list_tags. Nous utilisons également une combinaison de [**sleep()**](https://www.programiz.com/python-programming/time/sleep, 'doc de sleep') et de [**randint**](https://www.geeksforgeeks.org/python-randint-function/, 'doc de randint') pour être doux sur le site Web. (La fonction randint() choisira un entier aléatoire entre les limites supérieures et inférieures données, dans ce cas, **10** et **2**, respectivement, pour chaque itération de la boucle. Utilisation de la fonction randint() en combinaison avec la fonction **sleep()** aidera à ajouter des pauses courtes et aléatoires dans la vitesse d'exploration du programme. La fonction sleep() arrêtera l'exécution du programme pendant le nombre de secondes donné. Le nombre de secondes sera introduit de manière aléatoire dans la fonction sleep à l'aide de randint( ). Reportez-vous au code ci-dessous pour faciliter la compréhension)

In [76]:
# Maintenant nous avons un résumé au dessus de la fonction 
def get_item_list_tags():
    item_list_tags = []
    for page in range(1,5):
        items_url = f"https://www.alibaba.com/trade/search?fsb=y&IndexArea=product_en&CatId=&SearchText=Inflight+Items&viewtype=G&tab={page}"
        response = requests.get(items_url)
        page_contents = response.text
        if response.status_code != 200:
            raise Exception('Failed to load page {}'.format(items_url))
        doc = BeautifulSoup(page_contents, "html.parser")
        for item in doc.find_all("div", {'class': "organic-gallery-offer-outter J-offer-wrapper"}):
                item_list_tags.append(item)
        sleep(randint(2,10))
        print('Téléchargement du nombre de page', page)  
    return item_list_tags
item_list_tags = get_item_list_tags()
len (item_list_tags)

Téléchargement du nombre de page 1
Téléchargement du nombre de page 2
Téléchargement du nombre de page 3
Téléchargement du nombre de page 4


32

In [78]:
[t.text for t in item_list_tags]

['1/6Fashionable inflight items plastic sound insulation ear plugs$0.038500000.0 Sets(Min Order)10YRSCNSupplier4.8(2)|Contact Supplier',
 '1/6Wholesale Lowest Price China Children\'s Birthday Party Items Supplies$0.01100.0 Pieces(MOQ)10YRSCNSupplier4.9(40)|"excellent service"Contact Supplier',
 '1/3Birthday party items gift decorations supplies$3.00500.0 Sets(MOQ)4YRSCNSupplier4.6(4)|"good customer support"Contact Supplier',
 '1/6Airline Blanket / Inflight Blanket/ Polyester Fleece Blanket fire retardant Item 040217$1.30-$5.501.0 Pieces(MOQ)4YRSCNSupplier4.9(17)|"Fast delivery"Contact Supplier',
 '1/6Airline Catering Equipment Buffet Accessories Food Carts Trolleys$40.001.0 Units(MOQ)5YRSCNSupplierContact Supplier',
 '1/6SmallOrders RF19 Game console mobile phone handle TV handle Hot selling practical wireless game controller promotional gift$5.921 Unit(MOQ)1YRSCNSupplier4.5(10)|"great service"Contact Supplier',
 '1/6SmallOrders G020603 Pet SuppliesDog ToysPuzzle Leaking Food Molar Cle

>Si nous n'utilisons pas de boucle pour récupérer plus de pages du site Web, cela ne donne que 8 résultats. Dans ce cas; il y a deux options : nous pouvons simplement scraper plus de pages, avec 8 résultats (dans ce cas) dans chacune avec **Requests-Beautifulsoup** OU utiliser **Selenium**. Mais nous irons avec le premier choix puisque c'est facile.

Le code Web Scraping dépend de la structure de la page Web. Ainsi, si la structure change, votre code et les balises sélectionnés doivent changer/être mis à jour.

>**Remarque** : L'option correcte n'a pas besoin d'être uniquement la balise **div** sélectionnée ci-dessus. Il peut y avoir ou non plus d'une balise correcte pour obtenir les données requises après des essais et des erreurs minutieux. Notre objectif doit toujours être d'obtenir les données nécessaires de la meilleure façon possible.


## Obtenir les détails de la description du titre de l'élément

Après l'étape précédente d'extraction de tous les détails de l'élément, nous extrayons tous les titres requis dans un format de dictionnaire pour capturer la valeur nulle. (Nous pouvons également utiliser un bloc <code>try-except</code> qui est une instruction pour faire quelque chose sauf pour aucun cas, mais nous ferons de cette façon sur une autre instance sur ce même projet) Veuillez noter que **all_item_titledetail_list** peut être trouvé à partir de la balise **p** avec <code>class= 'elements-title-normal__content medium'</code> lors de l'inspection du site Web comme à l'étape précédente.

Par expérience, nous savons qu'une combinaison de dictionnaires peut être difficile à gérer lors de la création de blocs de données. Ainsi, après avoir obtenu les dictionnaires de données primaires, nous convertissons la liste de dictionnaires ci-dessus en une simple liste comme ci-dessous :

In [80]:
# maintenant, nous allons suivre les étapes ci-dessus pour obtenir les détails 
# de l'en-tête dans une petite fonction
def get_itemdetail(item_list_tags):
    all_item_titledetail_list = []
    for item in item_list_tags:
        title = item.find ('p', {'class': 'elements-title-normal__content medium'})
        all_item_titledetail_list.append({"Title":title.text.strip()})

    itemdetail=[]
    for tag in all_item_titledetail_list:
        itemdetail.append(tag.get('Title'))
    return itemdetail

itemdetail = get_itemdetail(item_list_tags)
len (itemdetail)#cela donnera de la longueur
itemdetail #cela listera les détails de l'en-tête

['Fashionable inflight items plastic sound insulation ear plugs',
 "Wholesale Lowest Price China Children's Birthday Party Items Supplies",
 'Birthday party items gift decorations supplies',
 'Airline Blanket / Inflight Blanket/ Polyester Fleece Blanket fire retardant Item 040217',
 'Airline Catering Equipment Buffet Accessories Food Carts Trolleys',
 'SmallOrders RF19 Game console mobile phone handle TV handle Hot selling practical wireless game controller promotional gift',
 'SmallOrders G020603 Pet SuppliesDog ToysPuzzle Leaking Food Molar Cleaner Elastic Molar Ball Custom promotional toys and gifts',
 'SmallOrders RF19 Hot selling practical wireless game controller game console mobile phone handle TV handle promotional gift',
 'Fashionable inflight items plastic sound insulation ear plugs',
 "Wholesale Lowest Price China Children's Birthday Party Items Supplies",
 'Birthday party items gift decorations supplies',
 'Airline Blanket / Inflight Blanket/ Polyester Fleece Blanket fire r

## Extracting the Item Price, Ratings, and URLs

Nous devrons répéter en utilisant des codes personnalisés pour toutes les paires clé-valeur requises, comme indiqué ci-dessous.

In [81]:
#nous résumerons les étapes ci-dessus pour obtenir le prix dans une fonction
def get_itemprice(item_list_tags):
    item_price_tags = []
    for item in item_list_tags:
        item_price = item.find ('span', {'class': 'elements-offer-price-normal__price'})
        if item_price == None:
            item_price_tags.append({"ItemPrice":item_price})
        else:
            item_price_tags.append({"ItemPrice":item_price.get_text(strip=True)})

    itemprice=[]
    for tag in item_price_tags:
        itemprice.append(tag.get('ItemPrice'))
    return itemprice

#this name "itemprice" should match with the name used in key:value pair value which will be used later in dict for df
itemprice = get_itemprice(item_list_tags)
len(itemprice)

32

Dans ce cas, le prix apparaît parfois comme **"None"** pour un article avec un prix d'offre, nous devons donc afficher le prix d'offre. (Pour certains autres articles, le prix de l'offre affichera une valeur **"None"** puisque le prix régulier est valide/disponible)

Nous créerons une fonction pour obtenir la note/les étoiles également comme dans le cas ci-dessus.

In [83]:
#nous allons créer une petite fonction pour effectuer les étapes ci-dessus
def get_Stars(item_list_tags):
    star_tags = []
    for item in item_list_tags:
        star = len(item.find_all('i', {'class': "iconfont iconzuanshi seller-star-level__dm dm-orange"}))
        star_tags.append({"Stars":star})
    star=[]
    for tag in star_tags:
        star.append(tag.get('Stars'))
    return star
star = get_Stars(item_list_tags)
star #cela listera les détails de l'étoile

[0,
 2,
 1,
 3,
 0,
 2,
 2,
 0,
 0,
 2,
 1,
 3,
 0,
 0,
 0,
 0,
 0,
 2,
 1,
 3,
 0,
 0,
 2,
 1,
 0,
 2,
 1,
 3,
 2,
 2,
 0,
 0]

Nous allons répéter les étapes ci-dessus mais les personnaliser pour obtenir l'URL comme ci-dessous.

In [84]:
#nous allons résumer les étapes ci-dessus pour obtenir des URL dans une petite fonction
def get_URLS(item_list_tags):
    website_tags = []
    base_url = 'https:'
    for item in item_list_tags:
        website = item.find ('a', {'class': 'organic-gallery-offer__img-section'})
        website_tags.append({"Website":base_url+website['href']})
    URLS=[]
    for tag in website_tags:
        URLS.append(tag.get('Website'))
    return URLS
URLS = get_URLS(item_list_tags)

In [ ]:
URLS

['https://www.alibaba.com/product-detail/Fashionable-inflight-items-plastic-sound-insulation_60383877450.html',
 'https://www.alibaba.com/product-detail/Wholesale-Lowest-Price-China-Children-s_60507339708.html',
 'https://www.alibaba.com/product-detail/Birthday-party-items-gift-decorations-supplies_60826373825.html',
 'https://www.alibaba.com/product-detail/Airline-Blanket-Inflight-Blanket-Polyester-Fleece_60753601852.html',
 'https://www.alibaba.com/product-detail/Airline-Catering-Equipment-Buffet-Accessories-Food_62393995548.html',
 'https://www.alibaba.com/product-detail/SmallOrders-RF19-Game-console-mobile-phone_1600438312626.html',
 'https://www.alibaba.com/product-detail/SmallOrders-G020603-Pet-SuppliesDog-ToysPuzzle-Leaking_1600341345104.html',
 'https://www.alibaba.com/product-detail/SmallOrders-RF19-Hot-selling-practical-wireless_1600436531873.html',
 'https://www.alibaba.com/product-detail/Fashionable-inflight-items-plastic-sound-insulation_60383877450.html',
 'https://www.al